In [11]:
import pygsti
import pygsti.algorithms.fiducialselection as fidsel
import pygsti.algorithms.germselection as germsel
from scipy.linalg import expm
import numpy as np
from pygsti.modelmembers import states, povms, operations as ops
from pygsti.models import modelconstruction as mc
from pygsti.modelmembers.operations import LindbladErrorgen
import sympy as sp
from pygsti.baseobjs import QubitSpace


In [12]:
class EEBasis():
    """
    Elementary error basis

    Parameters:
        -features: pauli transfer representations for different error processes
        -rates: sympy vector of rates
    """

    def __init__(self, num_qubits, gate_label=('G', 0), parameterization='HSCA'):
        """
        setup the feature dictionary with keys that are sympy variables
        """
        self.num_qubits = num_qubits
        self.label_map = dict()  # maps params to corresponding pygsti labels
        self.labels = LindbladErrorgen.from_error_generator(4 ** num_qubits).coefficient_labels()
        self.params = []
        for lbl in self.labels:
            if len(lbl.basis_element_labels) > 1:
                pstr1 = str()
                pstr2 = str()
                lbl1 = lbl.basis_element_labels[0]
                lbl2 = lbl.basis_element_labels[1]
                if len(lbl1) < num_qubits:
                    flg = 0
                    for i in range(num_qubits):
                        if i in lbl.support:
                            pstr1 += lbl1[flg]
                            flg += 1
                        else:
                            pstr1 += 'I'
                else:
                    pstr1 = lbl.basis_element_labels[0]
                if len(lbl2) < num_qubits:
                    flg = 0
                    for i in range(num_qubits):
                        if i in lbl.support:
                            pstr2 += lbl2[flg]
                            flg += 1
                        else:
                            pstr2 += 'I'
                else:
                    pstr2 = lbl.basis_element_labels[1]
                pstr = pstr1 + 'x' + pstr2
            else:
                pstr = str()
                flg = 0
                for idx in range(num_qubits):
                    if idx in lbl.support:
                        pstr += lbl.basis_element_labels[0][flg]
                        flg += 1
                    else:
                        pstr += 'I'
            if len(gate_label) > 1:
                gstr = str(gate_label[0]) + 'q' + str(gate_label[1])
            else:
                gstr = gate_label[0]
            param = sp.symbols(f'{{{lbl.errorgen_type}}}^{{{pstr}}}_{{{gstr}}} ')
            self.params.append(param)
            self.label_map[param] = lbl

    def __len__(self):
        return len(self.params)

    @property
    def vec(self):
        return list(self.params)

    @property
    def mat(self):
        mat = np.zeros((4 ** self.num_qubits, 4 ** self.num_qubits), dtype='O')
        for x in self.vec:
            mat += x * self.bmat(x)
        return sp.Matrix(mat)

    @property
    def basis(self):
        """

        :return:
        """
        return [egenlbl_to_mat(lbl, self.num_qubits) for lbl in self.labels]

    @property
    def dual_basis(self):
        return [egenlbl_to_dual(lbl, self.num_qubits) for lbl in self.labels]

    def bmat(self, param):
        """
        returns the basis matrix that corresponds to the param
        """
        return egenlbl_to_mat(self.label_map[param], self.num_qubits)

    def dual_bmat(self, param):
        return egenlbl_to_dual(self.label_map[param], self.num_qubits)

    def extract_rates(self, matrix):
        """
        extracts the rates of the given matrices
        """
        rates = dict()
        for lbl in tqdm(self.labels):
            dgen = egenlbl_to_dual(lbl, self.num_qubits)
            rates[lbl] = np.trace(dgen.conj().T @ matrix)
        return rates



In [22]:
# construct an initial model and set params to static

gs_model = mc.create_explicit_model_from_expressions( [0, 1],
    ['Gx0','Gy0', 'Gx1','Gy1'], [ "X(pi/2,0)", "Y(pi/2,0)", "X(pi/2,1)", "Y(pi/2,1)"] )

gs_model.set_all_parameterizations('H')
gs_model.num_params

90

In [23]:
# define the target cr operations

unitary_cr = expm(-1j * np.pi / 4 * pygsti.sigmazx)
mat_cr = pygsti.tools.unitary_to_superop(unitary_cr)

In [24]:
# define and print a 2-qubit elementary error basis

eebasis = EEBasis(2)
[(idx, e) for idx, e in enumerate(eebasis.vec)]

/home/jpmarceaux/anaconda3/envs/ErrorGenerators/lib/python3.10/site-packages/pygsti/modelmembers/operations/lindbladerrorgen.py:1691: UserWarning: Using 'CPTP' as a Lindblad type is deprecated, and you should now use 'CPTPLND'
  _warnings.warn("Using 'CPTP' as a Lindblad type is deprecated, and you should now use 'CPTPLND'")


[(0, {H}^{IX}_{Gq0}),
 (1, {H}^{IY}_{Gq0}),
 (2, {H}^{IZ}_{Gq0}),
 (3, {H}^{XI}_{Gq0}),
 (4, {H}^{XX}_{Gq0}),
 (5, {H}^{XY}_{Gq0}),
 (6, {H}^{XZ}_{Gq0}),
 (7, {H}^{YI}_{Gq0}),
 (8, {H}^{YX}_{Gq0}),
 (9, {H}^{YY}_{Gq0}),
 (10, {H}^{YZ}_{Gq0}),
 (11, {H}^{ZI}_{Gq0}),
 (12, {H}^{ZX}_{Gq0}),
 (13, {H}^{ZY}_{Gq0}),
 (14, {H}^{ZZ}_{Gq0}),
 (15, {S}^{IX}_{Gq0}),
 (16, {C}^{IXxIY}_{Gq0}),
 (17, {A}^{IXxIY}_{Gq0}),
 (18, {C}^{IXxIZ}_{Gq0}),
 (19, {A}^{IXxIZ}_{Gq0}),
 (20, {C}^{IXxXI}_{Gq0}),
 (21, {A}^{IXxXI}_{Gq0}),
 (22, {C}^{IXxXX}_{Gq0}),
 (23, {A}^{IXxXX}_{Gq0}),
 (24, {C}^{IXxXY}_{Gq0}),
 (25, {A}^{IXxXY}_{Gq0}),
 (26, {C}^{IXxXZ}_{Gq0}),
 (27, {A}^{IXxXZ}_{Gq0}),
 (28, {C}^{IXxYI}_{Gq0}),
 (29, {A}^{IXxYI}_{Gq0}),
 (30, {C}^{IXxYX}_{Gq0}),
 (31, {A}^{IXxYX}_{Gq0}),
 (32, {C}^{IXxYY}_{Gq0}),
 (33, {A}^{IXxYY}_{Gq0}),
 (34, {C}^{IXxYZ}_{Gq0}),
 (35, {A}^{IXxYZ}_{Gq0}),
 (36, {C}^{IXxZI}_{Gq0}),
 (37, {A}^{IXxZI}_{Gq0}),
 (38, {C}^{IXxZX}_{Gq0}),
 (39, {A}^{IXxZX}_{Gq0}),
 (40, {C}^{IXxZY}_

In [25]:
# associate the Hamiltonian params and define a reduced model
Hix = eebasis.vec[0]
Hiy = eebasis.vec[1]
Hiz = eebasis.vec[2]
Hxi = eebasis.vec[3]
Hxx = eebasis.vec[4]
Hxy = eebasis.vec[5]
Hxz = eebasis.vec[6]
Hyi = eebasis.vec[7]
Hyx = eebasis.vec[8]
Hyy = eebasis.vec[9]
Hyz = eebasis.vec[10]
Hzi = eebasis.vec[11]
Hzx = eebasis.vec[12]
Hzy = eebasis.vec[13]
Hzz = eebasis.vec[14]

all_ham_rates = [Hix, Hiy, Hiz, Hxi, Hxx, Hxy, Hxz, Hyi, Hyx, Hyy, Hyz, Hzi, Hzx, Hzy, Hzz]
reduced_ham_rates = [Hix, Hiy, Hiz, Hzi, Hzx, Hzy, Hzz]

In [26]:
# setup a model operation that is only parameterized by the Hamiltonian generators in the reduced model

model_egen_dict = {
    eebasis.label_map[h]: 0 for h in reduced_ham_rates
}

state_space = QubitSpace([0, 1]) # 2 qubits

model_generator = ops.LindbladErrorgen.from_elementary_errorgens(model_egen_dict, state_space=state_space)
model_op = ops.ExpErrorgenOp(model_generator)

In [27]:
# add the new model cr gate to the gate set model
gs_model.operations[('Gcr', 0, 1)] = model_op

In [28]:
gs_model.num_params

97

In [30]:
gs_model.num_nongauge_params

75

In [31]:
germs = pygsti.algorithms.germselection.find_germs(gs_model, randomize=False, algorithm='greedy', force=None)

Initial Length Available Germ List: 3409
Length Available Germ List After Deduping: 250
Length Available Germ List After Dropping Random Fraction: 250
Memory estimate of 0.1 GB for all-Jac mode.
Memory estimate of 0.0 GB for single-Jac mode.
Using greedy algorithm.
Constructed germ set:
['Gx0', 'Gy0', 'Gx1', 'Gy1', 'Gcr:0:1@(0,1)', 'Gx0Gy0Gx0Gx1Gy1Gx1', 'Gx0Gy0Gy0Gy1Gy1', 'Gx0Gx0Gy0Gx1Gy1Gy1']
Score: major=-52.0 minor=17.7541699797935, N: 52
